In [1]:
# import packages
import IPython
import ipywidgets
import ipyleaflet
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import numpy as np
import random
import os
import tensorflow as tf

import descarteslabs as dl
import descarteslabs.workflows as wf

### Create Geocontext (Currently not actually using any of this except the dates)
I was originally aiming to only compute the resulting mosaic over the AOI only using the geocontext but I binned it in the end and just let the interactive map constrain it to the AOI. Might be worth looking into further

In [2]:
with open("aoi_small_train.geojson") as f:
    fc = json.load(f)
    
aoi_geo = fc["features"][0]["geometry"]

aoi = wf.GeoContext(
    geometry = aoi_geo,
    crs="EPSG:3857",
    resolution = 10.0)

start_datetime = "2020-04-01"
end_datetime = "2020-07-28"

### Dilation Function
With Kernel

In [15]:
# Define simple functions for erosion and dilation
def erode_op(map_layer, iters, kernel):
    map_layer = ~map_layer
    for i in range(iters):
        map_layer = wf.conv2d(map_layer, kernel) > 0
    map_layer = ~map_layer 
    return map_layer

def dilate_op(map_layer, iters, kernel):
    for i in range(iters):
        map_layer = map_layer * 1.0
        map_layer = wf.conv2d(map_layer, kernel) > 0
    return map_layer

# Define a kernel and perform one erosion followed by two dilations
kernel = wf.Kernel(dims=(3,3), data=[0., 1., 0.,
                                      1., 1., 1.,
                                      0., 1., 0.])



### Sentinel 2 stack w/ cloud mask
Using whatever bands you like

In [70]:
sentinel_stack = (wf.ImageCollection.from_id("sentinel-2:L1C",
                                            start_datetime = start_datetime,
                                            end_datetime = end_datetime)
                  .pick_bands("red green blue cloud-mask red-edge-2 red-edge-3 red-edge-4 nir swir1 swir2")
                 )

sentinel_stack = sentinel_stack.filter(lambda img:img.properties["cloud_fraction"] <= 0.06)

dilated_mask = dilate_op(sentinel_stack.pick_bands("cloud-mask"), iters=75, kernel=kernel)
dilated_mask = dilated_mask.rename_bands("dilated-cloud-mask")

sentinel_stack = sentinel_stack.concat_bands(dilated_mask)

sentinel_masked = sentinel_stack.map(lambda img: img.mask(img.pick_bands('dilated-cloud-mask')==1))

### Create the image mosaic and compute over the AOI 

In [71]:
sentinel_mosaic = (sentinel_masked
                   .mosaic()
                   .pick_bands("red green blue"))

#sentinel_image = sentinel_mosaic.compute(aoi)

## Visualise the image
I think it looks pretty good, but there's room for improvement. Zoom out to see where it's not great (outside the city)

In [72]:
a_map = wf.interactive.MapApp()
a_map.center = (18.5577, -69.9145)  #18.5265, -69.9129
a_map.zoom = 15

sentinel_mosaic.visualize('image', scales=[(0, 1), (0, 1), (0, 1)], map=a_map)

a_map


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.
